In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import datetime
import numpy as np
import seaborn as sns
import re
import json
import pandas as pd
import math
import ast
import os
from collections import defaultdict
import sys
sys.path.append('..')
from utilities import logger
from utilities import configuration
from utilities import health_data

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier


In [2]:
config = configuration.get_config()
for key in config:
    print(f'{key:40} (type: {type(config[key])})')


logging = logger.init_logger(config['system_log'])
logging.debug('Logger has started ont notebook 09 Random sample of instances.ipynb ...')

system_log                               (type: <class 'str'>)
json_file                                (type: <class 'str'>)
train_val_json                           (type: <class 'str'>)
heldout_json                             (type: <class 'str'>)
unified_merged_file_cz                   (type: <class 'str'>)
unified_merged_file_noncz                (type: <class 'str'>)
unified_merged_file                      (type: <class 'str'>)
data_path                                (type: <class 'str'>)
cz_files                                 (type: <class 'list'>)
noncz_files                              (type: <class 'list'>)
2023-11-07 18:47:40,558 - root - DEBUG - Logger has started ont notebook 09 Random sample of instances.ipynb ...


In [3]:
f = open(config['train_val_json'])
train_val_data = json.load(f)
print(len(train_val_data))

all_admissions = []
for ix in train_val_data:
    all_admissions.append(
        health_data.Admission.from_dict_data(admit_id=int(ix), admission=train_val_data[ix])
        )
len(all_admissions)


# Dictionary organizing data by patient
patient2admissions = defaultdict(list)
for admission in all_admissions:
    code = admission.code
    patient2admissions[code].append(admission)

# Ordering patient list by discharge date (from back )
for patient_code in patient2admissions:
    admissions_list = patient2admissions[patient_code]
    admissions_list = sorted(admissions_list, key=lambda admission: admission.discharge_date, reverse=False)
    assert all([admissions_list[i].discharge_date <= admissions_list[i+1].discharge_date for i in range(len(admissions_list)-1)])
    patient2admissions[patient_code] = admissions_list
print(len(patient2admissions))

patient_count=0
valid_readmission_count=0
for patient_code in patient2admissions:
    patient_admissions = patient2admissions[patient_code]
    ix = 0 
    while ix < len(patient_admissions):
        readmission_code = patient_admissions[ix].readmission_code
        if health_data.ReadmissionCode.is_readmit(readmission_code):
            # Either is not the first admission (ix>0) or 
            # we don't have the patient previous admition (readmission close to begining of dataset) (admit-(2015-01-01))<28 days
            # assert ix>0 or (patient_admissions[ix].admit_date - datetime.datetime.fromisoformat('2015-01-01')).days<365
            if ix>0 and  patient_admissions[ix-1].is_valid_readmission(patient_admissions[ix]):
                patient_admissions[ix-1].add_readmission(patient_admissions[ix])
                valid_readmission_count+=1
        ix+=1
    patient_count+=1
valid_readmission_count

524986
263719


50533

Number of features=9929


9929

In [12]:
# freq = defaultdict(int)
# for admission in all_admissions:
#     for code in admission.diagnosis.codes:
#         freq[code]+=1
# print(len(freq))

9945


In [30]:

# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()
# type(vectorizer.fit_transform(['aaa aaab', 'aaa bs']))
# vectorizer.get_feature_names_out()

array(['aaa', 'aaab', 'bs'], dtype=object)

In [42]:
# from scipy.sparse._csr import csr_matrix

# def create_codes_matrix(admissions: list[health_data.Admission], freq_cutoff=0)->pd.DataFrame:
#     freq = defaultdict(int)
#     for admission in admissions:
#         for code in admission.diagnosis.codes:
#             freq[code.lower()]+=1
#     print(f'Found {len(freq)} distincts codes.')
#     codes_vocab = np.array([code for code in freq if freq[code]>freq_cutoff])
#     print(f'After filtering codes that appear {freq_cutoff} times or less we have {len(codes_vocab)} codes.')


#     return codes_vocab, matrix

# codes_vocab, matrix = create_codes_matrix(all_admissions)
# codes_vocab.shape

Found 9929 distincts codes.
After filtering codes that appear 0 times or less we have 9929 codes.


(9929,)

In [41]:
# from scipy.sparse._csr import csr_matrix


#     # freq = defaultdict(int)
#     # for admission in admissions:
#     #     for code in admission.diagnosis.codes:
#     #         freq[code]+=1
#     # print(f'Found {len(freq)} distincts codes.')
#     # codes_vocab = np.array([code for code in freq if freq[code]>freq_cutoff])
#     # print(f'After filtering codes that appear {freq_cutoff} times or less we have {len(codes_vocab)} codes.')
#     # code2index = dict([(code, idx) for idx,code in enumerate(codes_vocab)])
#     # del(freq)

#     # matrix = csr_matrix((len(admissions),len(codes_vocab)), dtype=np.int8)
#     # for ix, admission in enumerate(admissions):
#     #     for code in admission.diagnosis.codes:
#     #         if code in code2index:
#     #             matrix[ix, code2index[code]] += 1

#     return codes_vocab, matrix


# codes_vocab2, matrix = create_codes_matrix(all_admissions)
# codes_vocab2.shape

Number of features=9929


(9929,)

In [44]:
set(codes_vocab)==set(codes_vocab2)

True

In [32]:
matrix

<524986x8397 sparse matrix of type '<class 'numpy.float64'>'
	with 2314704 stored elements in Compressed Sparse Row format>

In [ ]:
# rng = np.random.default_rng(seed=5348363479653547918)

# train_indexes = rng.choice(range(len(all_admissions)),size=int(0.8*len(all_admissions)), replace=False)

# # Checking that every time I am getting the same training instances ( and validation instances)
# assert all(train_indexes[:3] ==np.array([478898, 46409, 322969]))
# assert all(train_indexes[-3:] ==np.array([415014, 330673, 338415]))
# assert hash(tuple(train_indexes))==2028319680436964623

# train_indexes = set(train_indexes)

# train = [admission for ix, admission in enumerate(all_admissions) if ix in train_indexes ]
# validation = [admission for ix, admission in enumerate(all_admissions) if not ix in train_indexes ]

# print(f'Size of training (before filter)=     {len(train):,}')
# print(f'Size of validation (before filter)=   {len(validation):,}')

# # Filtering missing values
# train = list(filter(lambda admission: not admission.has_missing, train))
# validation = list(filter(lambda admission: not admission.has_missing, validation))


# # Filtering STILL_BORNS admissions
# train = list(filter(lambda admission: admission.admit_category!=health_data.AdmitCategory.STILLBORN, train))
# validation = list(filter(lambda admission: admission.admit_category!=health_data.AdmitCategory.STILLBORN, validation))

# # Filtering CADAVER admissions
# train = list(filter(lambda admission: admission.admit_category!=health_data.AdmitCategory.CADAVER, train))
# validation = list(filter(lambda admission: admission.admit_category!=health_data.AdmitCategory.CADAVER, validation))


# print(f'Size of training (after filter)=      {len(train):,}')
# print(f'Size of validation (after filter)=    {len(validation):,}')

In [4]:
vocab, matrix = health_data.Admission.create_codes_matrix(all_admissions)

X = matrix
y = health_data.Admission.get_y(all_admissions)

from sklearn.model_selection import cross_validate

scoring = ['precision', 'recall', 'f1']
clf = SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(clf, X, y, scoring=scoring)


Number of features=9929


/Users/marianomaisonnave/Environments/alc/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
